<h1 style="color:#65AE11;">多 GPU</h1>

CUDA 可在单一主机上同时管理多部 GPU 设备。本节您将学习如何执行此操作。

<h2 style="color:#65AE11;">目标</h2>

完成本节内容的学习后，您将能理解并使用 CUDA 语法执行如下操作：

* 获取您的应用程序可使用的 GPU 的数量
* 激活任意可用的 GPU
* 在多个 GPU 上分配显存
* 在多个 GPU 上传入和转出显存数据
* 在多个 GPU 上启动核函数

<h2 style="color:#65AE11;">演示</h2>

运行以下单元以加载本节的视频演示。看完演示后，继续完成此 Notebook 余下的部分。

In [ ]:
%%html
<video width="800" controls>
  <source src="https://dli-lms.s3.us-east-1.amazonaws.com/assets/s-ac-02-v1-zh/05-multiple-gpus-01.mp4" type="video/mp4">
  Your browser does not support HTML video.
</video>

<h2 style="color:#65AE11;">获取多个 GPU 的相关信息</h2>

如要以运行程序的方式得出可用 GPU 的数量，请使用 `cudaGetDeviceCount`：

```c
int num_gpus;
cudaGetDeviceCount(&num_gpus);
```

如要以运行程序的方式得到当前处于活动状态的 GPU，请使用 `cudaGetDevice`：

```c
int device;
cudaGetDevice(&device); // `device` is now a 0-based index of the current GPU.
```

<h2 style="color:#65AE11;">设置当前的 GPU</h2>

对于每个主机线程，每次只有一个 GPU 设备处于活动状态。如要将特定的 GPU 设置为活动状态，请使用 `cudaSetDevice` 以及所需 GPU 的索引（从 0 开始）：

```c
cudaSetDevice(0);
```

<h2 style="color:#65AE11;">遍历可用的 GPU</h2>

一种常见模式为遍历可用的 GPU，并为每个 GPU 执行相应的操作：

```c
int num_gpus;
cudaGetDeviceCount(&num_gpus);

for (int gpu = 0; gpu < num_gpus; gpu++) {

    cudaSetDevice(gpu);
    
    // Perform operations for this GPU.
}    
```

<h2 style="color:#65AE11;">为多个 GPU 执行数据分块</h2>

在使用多个非默认流执行复制和计算重叠时，多个 GPU 中的每个 GPU 都可处理一个数据块。我们将创建和利用数据指针数组为每个可用的 GPU 分配显存：

```c
const int num_gpus;
cudaGetDeviceCount(&num_gpus);

const uint64_t num_entries = 1UL << 26;
const uint64_t chunk_size = sdiv(num_entries, num_gpus);

uint64_t *data_gpu[num_gpus]; // One pointer for each GPU.

for (int gpu = 0; gpu < num_gpus; gpu++) {

    cudaSetDevice(gpu);

    const uint64_t lower = chunk_size*gpu;
    const uint64_t upper = min(lower+chunk_size, num_entries);
    const uint64_t width = upper-lower;

    cudaMalloc(&data_gpu[gpu], sizeof(uint64_t)*width); // Allocate chunk of data for current GPU.
}
```

<h2 style="color:#65AE11;">为多个 GPU 复制数据</h2>

通过使用相同的遍历和分块技术，我们可在多个 GPU 上传入和传出数据：

```c
// ...Assume data has been allocated on host and for each GPU

for (int gpu = 0; gpu < num_gpus; gpu++) {

    cudaSetDevice(gpu);

    const uint64_t lower = chunk_size*gpu;
    const uint64_t upper = min(lower+chunk_size, num_entries);
    const uint64_t width = upper-lower;

    // Note use of `cudaMemcpy` and not `cudaMemcpyAsync` since we are not
    // presently using non-default streams.
    cudaMemcpy(data_gpu[gpu], data_cpu+lower, 
           sizeof(uint64_t)*width, cudaMemcpyHostToDevice); // ...or cudaMemcpyDeviceToHost
}
```

<h2 style="color:#65AE11;">为多个 GPU 启动核函数</h2>

通过使用相同的遍历和分块技术，我们可在多个 GPU 上启动核函数以处理数据块：

```c
// ...Assume data has been allocated on host and for each GPU

for (int gpu = 0; gpu < num_gpus; gpu++) {

    cudaSetDevice(gpu);

    const uint64_t lower = chunk_size*gpu;
    const uint64_t upper = min(lower+chunk_size, num_entries);
    const uint64_t width = upper-lower;

    kernel<<<grid, block>>>(data_gpu[gpu], width); // Pass chunk of data for current GPU to work on.
}
```

<h2 style="color:#65AE11;">知识检查</h2>

请回答下列问题，确认您已了解本节的主要目标。您可以通过单击问题下方的`...`单元来显示每个问题的答案。

---

**哪个 CUDA 运行时调用可以得到可用 GPU 的数量？**

1. `cudaGetDevice`
2. `cudaSetDevice`
3. `cudaGetDeviceCount`
4. `cudaGetDeviceProperties`

**答案：3**

---

**哪个 CUDA 运行时调用可选择当前让某个 GPU 处于活动状态？**

1. `cudaGetDevice`
2. `cudaSetDevice`
3. `cudaGetDeviceCount`
4. `cudaGetDeviceProperties`

**答案：2**

---

**单 GPU 系统上的 GPU 索引是多少？**

1. 1
2. 0

**答案：2**

---

<h2 style="color:#65AE11;">下一步</h2>

现在你已熟悉有关使用多个 GPU 的语法和技术，下一节中，你将运用对所学内容的理解重构基准密码应用，以使用多个 GPU。

请继续学习下一节：[*使用多个 GPU*](../06_Exercise_MGPU/Exercise_MGPU.ipynb)。

<h2 style="color:#65AE11;">可选的进一步学习的材料</h2>

以下内容供有时间和兴趣的学生就与本课程相关的主题进一步学习。

* 在上面，我们使用[深度优先](https://www.geeksforgeeks.org/difference-between-bfs-and-dfs/#:~:text=BFS(Breadth%20First%20Search)%20uses,edges%20from%20a%20source%20vertex.)的方法将一部分工作传递给每个GPU。在某些情况下，尤其是在数据量极高的情况下，使用宽度优先的方法可能更有意义。这种方法上的改变并不是需要额外的CUDA知识。不过，此[stack overflow的回答](https://stackoverflow.com/questions/11673154/concurrency-in-cuda-multi-gpu-executions)提供了一些使用深度优先和宽度优先方法的CUDA代码示例。
* 多个GPU之间进行对等内存传输，以及在多个节点上使用多个GPU，两者均不在本课程的讨论范围之内。 [此超级计算会议演示文稿](https://www.nvidia.com/docs/IO/116711/sc11-multi-gpu.pdf)将为您探索这些主题（以及更多）提供一个很好的起点。